# Correr experimentos
En este archivo está el código para correr los experimentos y escribir los resultados en archivos CSV.
> Los archivos se guardan en la carpeta _resultados_.

In [27]:
import math, subprocess
import pandas as pd
import numpy as np
import random, math
import os
import shutil
from IPython.display import display, clear_output
import time
import matplotlib.pyplot as plt

iteraciones = 5000
eps = 10e-14
alpha = 10e14
experimentos = []

# CN = Canonico
# RN = Random
# TN = Todos Unos
iniciales = ["CN", "RN", "TN"]

# Titulos de distintos CSV
columnas = ["Caso","Path","GSM","GSM-ERR","GSS","GSS-ERR","JM","JM-ERR","JS","JS-ERR","LU","Tiempo","n"];

## Metodos:
- Jacobi Formula
- Jacobi Matriz
- Gauss Seidel Formula
- Gauss Seidel Matriz
- LU Eigen

In [2]:
# Lectura de Matriz
def leer_matriz(path_instancia):
    A = np.loadtxt(path_instancia, skiprows=1, dtype='d')
    return A

In [3]:
def agregar_experimentos(dataframe):
    counter = 0
    while counter < len(dataframe):
        fila = dataframe.iloc[counter]
        experimentos.append([fila["dataset"], fila["archivo-A"], fila["archivo-x"], fila["archivo-b"], fila["n"]])
        counter = counter + 1

In [4]:
def ejecutar_metodo(pathA, pathB, pathX, caso, iteraciones, eps_M, alpha_M, ini):
    process = subprocess.Popen(
        ["./tp3", pathA, pathB, pathX, caso, str(iteraciones), str(eps_M), str(alpha_M), ini], 
        stderr=subprocess.PIPE, 
        stdout=subprocess.PIPE, 
        stdin=subprocess.PIPE, 
        universal_newlines = True)
    exit_code = process.wait()
    if exit_code != 0: 
        print(F"Hubo un error en la experimentacion con la instancia {pathA} - ${pathB} - ${pathX}")
    process.stdin.close();
    process.stdout.close();
    process.stderr.close();

In [5]:
def limpiador(path):
    if os.path.exists(path):
        for filename in os.listdir(path):
            if('.out' in filename):
                file_path = os.path.join(path, filename)
                try:
                    os.unlink(file_path)
                except Exception as e:
                    print('Failed to delete %s. Reason: %s' % (file_path, e))

In [6]:
def copy_paste(path, path_second):
    if not os.path.exists(path_second):
        os.mkdir(path_second)
    
    files = os.listdir(path)
    for filename in files:
        if('.out' in filename):
            file_path = os.path.join(path, filename)
            file_path_sec = os.path.join(path_second, filename)
            shutil.move(file_path, file_path_sec)

### Ejecución

In [28]:
dfs={}
casos=["edd-tiempos"]
experimentos = []
for caso in casos:
    dfs[caso]=pd.read_csv(F"instancias/{caso}/indice.csv")

for caso in casos:
    agregar_experimentos(dfs[caso])
    
if(not os.path.isdir("resultados")):
    os.mkdir("resultados")

### Primer Instancia
Tomaremos la matríz de 500x500 Estrictamente Diagonal Dominante y la ejecutamos para todos los vectores iniciales 100 veces.

In [23]:
experimento = experimentos[6]

In [24]:
filas = []

pathA = F"instancias/{experimento[0]}/{experimento[1]}"
pathX = F"instancias/{experimento[0]}/{experimento[2]}"
pathB = F"instancias/{experimento[0]}/{experimento[3]}"

for it in range(0, 5):
    
    clear_output(wait=True)
    display('Experimento: ' + str(it) + "/" + str(4))

    for ini in iniciales:
        ejecutar_metodo(pathA, pathX, pathB, F"instancias/{experimento[0]}/{experimento[0]}-{(pathX.split('-')[1])}-{ini}", iteraciones, eps, alpha, ini)

        filas.append([
            experimento[0],
            F"instancias/{experimento[0]}-VI-it-{it}/{experimento[0]}-{(pathX.split('-')[1])}",
            F"-{ini}-gsm.out",
            F"-{ini}-gsm-err.out",
            F"-{ini}-gss.out",
            F"-{ini}-gss-err.out",
            F"-{ini}-jm.out",
            F"-{ini}-jm-err.out",
            F"-{ini}-js.out",
            F"-{ini}-js-err.out",
            F"-{ini}-lu.out",
            F"-{ini}-tiempo.out",
            experimento[4]
        ]);

    copy_paste(F"instancias/{experimento[0]}", F"instancias/{experimento[0]}-VI-it-{it}") 

df_resultado = pd.DataFrame(filas, columns=columnas);
df_resultado.to_csv(F"resultados/resultado-it-VI.csv", index=False, header=True);

'Experimento: 0/4'

KeyboardInterrupt: 

In [25]:
filas_aux = filas[0:len(filas)-1]

In [26]:
df_resultado = pd.DataFrame(filas_aux, columns=columnas);
df_resultado.to_csv(F"resultados/resultado-it-VI.csv", index=False, header=True);

In [27]:
#copy_paste("./instancias/edd", "./instancias/edd-it-1")

In [28]:
limpiador("./instancias/edd")
limpiador("./instancias/sdp")
limpiador("./instancias/random")

In [30]:
# Experimentación con EG_Pivot y EG_Trig
columnas = ["Caso","Path","GSM","GSM-ERR","GSS","GSS-ERR","JM","JM-ERR","JS","JS-ERR","LU","LU-ERR","Tiempo","n"];

for it in range(0, 10):
    filas = []
    numero = 1
    for experimento in experimentos:
        # Voy mostrando que experimento se esta ejecutando.
        clear_output(wait=True)
        display('Iteracion: ' + str(it) + "/" + str(9))
        display('Experimento: ' + str(numero) + "/" + str(len(experimentos)))
        numero += 1

        pathA = F"instancias/{experimento[0]}/{experimento[1]}"
        pathX = F"instancias/{experimento[0]}/{experimento[2]}"
        pathB = F"instancias/{experimento[0]}/{experimento[3]}"

        ejecutar_metodo(pathA, pathX, pathB, F"instancias/{experimento[0]}/{experimento[0]}-{numero - 1}", iteraciones, eps, alpha, "TN")

        filas.append([
            experimento[0], 
            F"instancias/{experimento[0]}-it-{it}/{experimento[0]}-{(pathX.split('-')[1])}",
            F"-gsm.out",
            F"-gsm-err.out",
            F"-gss.out",
            F"-gss-err.out",
            F"-jm.out",
            F"-jm-err.out",
            F"-js.out",
            F"-js-err.out",
            F"-lu.out",
            F"-lu-err.out",
            F"-tiempo.out",
            experimento[4]
        ]);

        df_resultado = pd.DataFrame(filas, columns=columnas);
        df_resultado.to_csv(F"resultados/resultado-it-{it}.csv", index=False, header=True);
        
    copy_paste(F"instancias/{experimento[0]}", F"instancias/{experimento[0]}-it-{it}")    

'Iteracion: 9/9'

'Experimento: 9/9'